MusicLang2 : Controllable Symbolic Music Generation
========================================================


![picture](https://drive.google.com/uc?id=1uREq4YCxYeGt-KLL7NBBZoivE4OiJw0V)


🎶  &nbsp; You want to generate music that you can export to your favourite DAW in MIDI ?
-------------

🎛️ &nbsp; You want to control the chord progression of the generated music ?
-------------

🚀  &nbsp; You need to run it fast on your laptop without a gpu ?
----------------


Here is MusicLang2, your controllable music copilot.
========


Feel the magic of AI generated music with the demo below.


<i>You want to integrate this model in your music software/application ? </i>[Contact us](https://www.musiclang.io/contact)


In [1]:
#@title Setup - Run this first then run a cell in any order

# Installing the package and setup Audio to listen to our generated music
# For the time being we are still on development mode :)
%%capture
!pip install musiclang-predict

# The audio stuff
!pip install pyFluidSynth &> /dev/null
!apt install ffmpeg
!apt install fluidsynth &> /dev/null
!pip install midi2audio &> /dev/null
!cp /usr/share/sounds/sf2/FluidR3_GM.sf2 ./font.sf2 &> /dev/null

def play_music(file):
  from midi2audio import FluidSynth
  import os
  FluidSynth("font.sf2").midi_to_audio(file, 'result.wav')
  from IPython.display import Audio, display
  !ffmpeg -i result.wav -acodec libmp3lame -y -loglevel quiet -stats result.mp3
  wn = Audio(filename="result.mp3", autoplay=False)
  display(wn)

In [2]:
#@title Free generation of musical ideas
import time
from musiclang_predict import MusicLangPredictor

# @markdown Generate a free music idea
nb_tokens = 1024 #@param {type:"slider", min:32, max:1024, step:32}
temperature = 0.9 #@param {type:"slider", min:0.0, max:1, step:0.05}
top_p = 1.0 #@param {type:"slider", min:0.0, max:1.0, step:0.05}
seed = 16 #@param {type: "integer", "info":"aez"}

ml = MusicLangPredictor('musiclang/musiclang-v2')

start = time.time()
score = ml.predict(
    nb_tokens=nb_tokens,  # 1024 tokens ~ 25s of music (depending of the number of instruments)
    temperature=temperature,
    topp=top_p,
    rng_seed=seed # change here to change result, or set to 0 to unset seed
)
end = time.time()
print(f"Generated in {end - start} seconds")


score.to_midi('test.mid')
play_music('test.mid')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.bin:   0%|          | 0.00/255k [00:00<?, ?B/s]

model.bin:   0%|          | 0.00/247M [00:00<?, ?B/s]

tokenizer-base.json:   0%|          | 0.00/566 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/18.7k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/973 [00:00<?, ?B/s]

Generated in 50.47602725028992 seconds
size=     345kB time=00:00:21.99 bitrate= 128.4kbits/s speed=36.5x    


In [3]:
#@title Control the chord progression of a free generation
import time
from musiclang_predict import MusicLangPredictor

# Control the chord progression here
# @markdown Control the chord progression
# @markdown <i>chord qualities : M, m, 7, m7b5, sus2, sus4, m7, M7, dim, dim0</i>.
# @markdown You can also specify the bass if it belongs to the chord (eg : Bm/D)
chord_progression = "Am CM Dm E7 Am" #@param {type: "string" }
nb_tokens = 1024 #@param {type:"slider", min:32, max:1024, step:32}
temperature = 0.8 #@param {type:"slider", min:0.0, max:1, step:0.05}
top_p = 1.0 #@param {type:"slider", min:0.0, max:1.0, step:0.05}
seed = 42 #@param {type: "integer", "info":"aez"}


ml = MusicLangPredictor('musiclang/musiclang-v2')

start = time.time()
score = ml.predict_chords(
    chord_progression,
    time_signature=(4, 4),
    temperature=temperature,
    topp=top_p,
    rng_seed=seed # set to 0 to unset seed
)

end = time.time()
print(f"Generated in {end - start} seconds")

score.to_midi('test.mid', tempo=110, time_signature=(4, 4))
play_music('test.mid')

/usr/local/lib/python3.10/dist-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/usr/local/lib/python3.10/dist-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/usr/local/lib/python3.10/dist-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/usr/local/lib/python3.10/dist-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


Generated in 6.28737735748291 seconds
size=     171kB time=00:00:10.89 bitrate= 128.8kbits/s speed=  28x    


In [4]:
#@title Use a prompt to continue your own music !
import time
from musiclang_predict import MusicLangPredictor, corpus


song_name = 'mozart_alla_turca' #@param ['bach_847', 'bob_marley_jammin', 'boney_m_ma_baker', 'mozart_alla_turca', 'white_stripes_seven_nation_army']
nb_tokens = 1504 #@param {type:"slider", min:32, max:2048, step:32}
temperature = 0.85 #@param {type:"slider", min:0.0, max:1, step:0.05}
top_p = 1.0 #@param {type:"slider", min:0.0, max:1.0, step:0.05}
seed = 1000 #@param {type: "integer", "info":"seed"}

ml = MusicLangPredictor('musiclang/musiclang-v2')

start = time.time()
score = ml.predict(
    score=corpus.get_midi_path_from_corpus(song_name),
    nb_tokens=nb_tokens,
    prompt_chord_range=(0,4),
    temperature=temperature,
    topp=top_p,
    rng_seed=seed # set to 0 to unset seed
)

end = time.time()

print(f"Generated in {end - start} seconds")

score.to_midi('test.mid', tempo=120, time_signature=(4, 4))
play_music('test.mid')


Generated in 85.26873445510864 seconds
size=     477kB time=00:00:30.48 bitrate= 128.3kbits/s speed=36.1x    


In [ ]:
#@title Combine both a prompt and a chord progression !
import time
from musiclang_predict import MusicLangPredictor, corpus


song_name = 'bach_847' #@param ['bach_847', 'bob_marley_jammin', 'boney_m_ma_baker', 'mozart_alla_turca', 'white_stripes_seven_nation_army']
# @markdown <i>chord qualities : M, m, 7, m7b5, sus2, sus4, m7, M7, dim, dim0</i>.
# @markdown You can also specify the bass if it belongs to the chord (eg : Bm/D)
chord_progression = "Cm C7/E Fm F#dim G7 Cm" #@param {type: "string" }
nb_tokens = 1024 #@param {type:"slider", min:32, max:1024, step:32}
temperature = 0.8 #@param {type:"slider", min:0.0, max:1, step:0.05}
top_p = 1.0 #@param {type:"slider", min:0.0, max:1.0, step:0.05}
seed = 3666 #@param {type: "integer", "info":"aez"}

ml = MusicLangPredictor('musiclang/musiclang-v2')

start = time.time()
score = ml.predict_chords(
    chord_progression,
    score=corpus.get_midi_path_from_corpus(song_name),
    time_signature=(4, 4),
    nb_tokens=1024,
    prompt_chord_range=(0,4),
    temperature=temperature,
    topp=top_p,
    rng_seed=seed # set to 0 to unset seed
)

end = time.time()

print(f"Generated in {end - start} seconds")

score.to_midi('test.mid', tempo=110, time_signature=(4, 4))
play_music('test.mid')


Generated in 25.28744411468506 seconds
size=     342kB time=00:00:21.81 bitrate= 128.4kbits/s speed=11.9x    
